<a href="https://colab.research.google.com/github/akaicomet/NJOITOpenDataCenter-Pension-Record/blob/master/NJOITOpenDataCenter_Pension_Record_ModelEaluating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install tqdm
!pip install scikit-learn
!pip install sodapy
!pip install matplotlib
!pip install hyperopt
!pip install bayesian-optimization
!pip install tbvaccine 
!pip install MulticoreTSNE
!pip install -U -q PyDrive

#1. Copy python files from  GItHub repository to local

In [2]:
!git clone https://github.com/akaicomet/NJOITOpenDataCenter.git
!git clone https://github.com/akaicomet/akaicomet_Utility.git
!git clone https://github.com/akaicomet/akaicomet_Chart.git
!git clone https://github.com/akaicomet/akaicomet_ML.git

fatal: destination path 'NJOITOpenDataCenter' already exists and is not an empty directory.
fatal: destination path 'akaicomet_Utility' already exists and is not an empty directory.
fatal: destination path 'akaicomet_Chart' already exists and is not an empty directory.
fatal: destination path 'akaicomet_ML' already exists and is not an empty directory.


#2. Run .py files in local

In [0]:
%run -i /content/akaicomet_Utility/ErrorHandling.py

%run -i /content/NJOITOpenDataCenter/YourMoneyActivePensionMembers.py 
%run -i /content/NJOITOpenDataCenter/YourMoneyRetiredPensionMembers.py
%run -i /content/NJOITOpenDataCenter/YourMoneyPensionMembersMerge.py

%run -i /content/akaicomet_Chart/X-Ray_Scan.py
%run -i /content/akaicomet_Chart/akaicomet_Chart.py

%run -i /content/akaicomet_ML/ML_Init.py
%run -i /content/akaicomet_ML/ML_Auto.py
%run -i /content/akaicomet_ML/AutoFeatureEngineering.py


#3. Import libraries

In [0]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import random
from sodapy import Socrata
#from pandas.plotting import scatter_matrix
#from datetime import datetime
import seaborn as sb
from tqdm import tqdm
import warnings
from textwrap import wrap

#4. YourMoney and the NJOIT Open Data Center 
##4.1 YourMoney Active Pension Members API

**Pension member data load from NJOIT** 

In [5]:
client = Socrata("data.nj.gov", None)
results = client.get("45bd-gwii", limit=100000000)#100000000

**Data Cleansing and Preparation for YourMoney Active Pension Members data**

In [0]:
ActivePension_df = ActivePensionDataWrangler(results)

##4.2 YourMoney Retired Pension Members API
**Pension payment member data load from NJOIT**

In [0]:
results = client.get("8up4-62p6",   limit=100000000)

**Data Cleansing and Preparation for YourMoney Retired Pension data**

In [0]:
RetiredPension_df_grped = RetiredPensionMembersDataWrangler(results)

**Merge YourMoney Active Pension Members data and YourMoney Retired Pension data**

In [0]:
ActivePension_df = PensionMembersMerge(ActivePension_df,RetiredPension_df_grped)
del results

# 5 Recheck  Correlation Variable Validation:

In [17]:
#x_cols2 = ['_20_year_status',	'all_employers_salary_amt',		'employer_freq_pensioned', 'total_months_qty',	'location_freq_pensioned',		'current_employer_salary_rollingamt', 	'service_months_qty',	'pension_freq_pensioned', 'pension_fund_name_enc', 'pension_group_name_enc' ]
x_cols2 = ['_20_year_status','all_employers_salary_amt','employer_freq_pensioned','total_months_qty','location_freq_pensioned','service_months_qty','pension_freq_pensioned','pension_fund_name_enc','pension_group_name_enc' ]
ActivePension_df[x_cols2].corr()

,_20_year_status,all_employers_salary_amt,employer_freq_pensioned,total_months_qty,location_freq_pensioned,service_months_qty,pension_freq_pensioned,pension_fund_name_enc,pension_group_name_enc
_20_year_status,1.000000,0.295118,-0.074679,0.844411,0.082618,-0.002429,0.091064,-0.238068,-0.113550
all_employers_salary_amt,0.295118,1.000000,-0.066198,0.395345,-0.030279,-0.027708,-0.439474,0.207998,0.097105
employer_freq_pensioned,-0.074679,-0.066198,1.000000,-0.065524,0.187599,0.026186,0.040389,0.099662,-0.033238
total_months_qty,0.844411,0.395345,-0.065524,1.000000,0.067168,0.000102,0.047303,-0.187378,-0.121364
location_freq_pensioned,0.082618,-0.030279,0.187599,0.067168,1.000000,0.056022,0.308507,0.047458,-0.063150
service_months_qty,-0.002429,-0.027708,0.026186,0.000102,0.056022,1.000000,0.079330,0.052583,-0.034342
pension_freq_pensioned,0.091064,-0.439474,0.040389,0.047303,0.308507,0.079330,1.000000,-0.306339,-0.308736
pension_fund_name_enc,-0.238068,0.207998,0.099662,-0.187378,0.047458,0.052583,-0.306339,1.000000,0.185796
pension_group_name_enc,-0.113550,0.097105,-0.033238,-0.121364,-0.063150,-0.034342,-0.308736,0.185796,1.000000


#6 Initial Basic Model Selected 

In [18]:
y_train = ActivePension_df['IsPensionPaid']
X_train = ActivePension_df[x_cols2]

X_train = X_train.astype(float)

clfs = dict()
clfs_score = dict()

scaling_def = {'QuantileTransformer_n':QuantileTransformer(output_distribution='normal')}
redemensioning_def = {'PCA': PCA(n_components=0.99, svd_solver='full')}

for data in map(lambda x:Redimensioning2(x[1],x[0],redemensioning_def), scaling3(X_train,scaling_def)):
  for x in data:
    MLModelBuilding('RandomForestClassifier', x[1], x[0], y_train, 0.3, clfs, [0.1, 1],clfs_score)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Train score: 0.9907248539102325
Test score: 0.9546272120684653
Test2 score: 0.9814911517261387
RandomForestClassifier|PCA QuantileTransformer_n|



In [13]:
clfs['RandomForestClassifier|PCA QuantileTransformer_n|']

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

#7 Forecast Model Review

In [0]:
scaling_def = {'QuantileTransformer_n':QuantileTransformer(output_distribution='normal')}
redemensioning_def = {'PCA': PCA(n_components=0.99, svd_solver='full')}

sample = ActivePension_df[x_cols2 + ['IsPensionPaid']].sample(n=50000, random_state=5)

for data in map(lambda x:Redimensioning2(x[1],x[0],redemensioning_def), scaling3(sample[x_cols2],scaling_def)):
  for x in data:
    clf_pred = clfs['RandomForestClassifier|PCA QuantileTransformer_n|'].predict(x[0])

In [0]:
Predict_df = sample
Predict_df['Predict'] = clf_pred

In [31]:
Predict_df[Predict_df['IsPensionPaid'] != Predict_df['Predict']]

,_20_year_status,all_employers_salary_amt,employer_freq_pensioned,total_months_qty,location_freq_pensioned,service_months_qty,pension_freq_pensioned,pension_fund_name_enc,pension_group_name_enc,IsPensionPaid,Predict
35075,1,83030.0,54.0,333,5790,9,4551,0.767324,0.479019,1.0,0.0
28603,1,51781.0,15.0,358,1677,10,16480,0.441019,0.448616,1.0,0.0
17929,1,73519.0,14.0,454,5790,10,16480,0.441019,0.448616,1.0,0.0
57205,1,34715.0,60.0,529,1421,1,16480,0.441019,0.448616,1.0,0.0
6054,1,78708.0,1.0,375,205,3,5243,0.378994,0.479019,1.0,0.0
23517,1,128000.0,57.0,438,5790,6,16480,0.441019,0.448616,1.0,0.0
5108,1,34612.0,405.0,388,2309,4,16480,0.441019,0.448616,1.0,0.0
33500,1,201518.0,29.0,411,1371,3,5243,0.378994,0.479019,0.0,1.0
45852,1,73924.0,195.0,346,5790,10,16480,0.441019,0.448616,1.0,0.0
44544,1,84279.0,50.0,449,5790,5,16480,0.441019,0.448616,1.0,0.0
